# What You’ll Learn
- Query free stock market history data instantly
- Run a Python faker function to create 100 synthetic traders
-Size up compute to create 3 billion synthetic trades
- Insert those trades ordered by trader, symbol, and date so filtered queries are significantly faster
- Create a cluster key to future proof that trade table from any non-sorted data
- Create a window function to calculate real time trades, cash, and Profit and Loss (PnL)
- Query those 3 billion rows with only xsmall compute and 3 second run-time
- See Snowflake's 3 caches in action
- Database-level Zero Copy Clone to create an instant Dev environment
- Time Travel to see and roll back up to 90 days of data
- Drop and Undrop Databases and Tables

# What You’ll Need
A Snowflake account: A free trial on Enterprise Edition will suffice.

In [ ]:
# Import python packages
import streamlit as st
# import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


- Mount this Share [Cybersyn Financial & Economic Essentials](https://app.snowflake.com/marketplace/listing/GZTSZAS2KF7/cybersyn-financial-economic-essentials) as a database named Financial__Economic_Essentials (Notice 2 underscores after Financial).
- Grant the PUBLIC role to access that database.
- Click Get

In [ ]:
--setup
    use role finservam_admin;
    use warehouse finservam_devops_wh;
    use schema finservam.public;

--Verify Data Marketplace Share
    select *
    from Financial__Economic_Essentials.cybersyn.stock_price_timeseries
    where ticker = 'SNOW' and variable = 'post-market_close' order by date;


In [ ]:
//Create role
    use role accountadmin;
    create role if not exists finservam_admin comment = 'Ownership of finservam database and demo';

    //Create compute
    create warehouse if not exists finservam_devops_wh
        with warehouse_size = 'xsmall' auto_suspend = 120 initially_suspended = true comment = 'Financial Services DevOps Compute';
    create warehouse if not exists xsmall_const_wh
        with warehouse_size = 'xsmall' auto_suspend = 60 initially_suspended = true comment = 'Constant so should always be XS and not resized';
        


//Permissions are granular to meet your requirements
    create database if not exists finservam comment = 'Financial Service Asset Management';
    
    grant ownership on database finservam to role finservam_admin;
    grant ownership on schema finservam.public to role finservam_admin;
    
    grant ownership on warehouse finservam_devops_wh to role finservam_admin;
    
    grant ownership on warehouse xsmall_const_wh to role sysadmin;
    grant monitor, operate, usage on warehouse xsmall_const_wh to role finservam_admin;
    
    grant role finservam_admin to role sysadmin;

    use schema finservam.public;

    create schema if not exists transform comment = 'for silver layer transformations in a medallion architecture.  generally not meant for end user visibility';
    grant ownership on schema transform to role finservam_admin;

    use schema finservam.public;
    use warehouse finservam_devops_wh;


In [ ]:
use role accountadmin;
grant role finservam_admin to role datascientist;

In [ ]:
--transform.exclude_symbol
    create or replace table transform.exclude_symbol
        comment = 'Exclude Symbols that have ever had a price less than 1 cent or greater than $4500'
    as
        select distinct ticker symbol
        from Financial__Economic_Essentials.cybersyn.stock_price_timeseries
        where
            variable = 'post-market_close'
            and primary_exchange_name in ('NASDAQ CAPITAL MARKET', 'NEW YORK STOCK EXCHANGE')
            and (value < .01 or value > 4500)
        order by 1;



In [ ]:
--stock_history
    create or replace transient table finservam.public.stock_history
        comment = 'Daily closing prices for NASDAQ & NYSE'
    as
    select
        ticker symbol,
        date,
        value close,
        primary_exchange_code exchange,
        asset_class
    from Financial__Economic_Essentials.cybersyn.stock_price_timeseries k
    left outer join transform.exclude_symbol e on e.symbol = k.ticker
    where 
        variable = 'post-market_close'
        and primary_exchange_name in ('NASDAQ CAPITAL MARKET', 'NEW YORK STOCK EXCHANGE')
    order by symbol, date;
    
    comment on column stock_history.close is 'security price at the end of the financial market business day';


In [ ]:
--Python fake function used to generate synthetic traders
create or replace function fake_py(locale varchar,provider varchar,parameters variant)
returns variant
language python
volatile
runtime_version = '3.8'
packages = ('faker','simplejson')
handler = 'fake'
as
$$
import simplejson as json
from faker import Faker
def fake(locale,provider,parameters):
  if type(parameters).__name__=='sqlNullWrapper':
    parameters = {}
  fake = Faker(locale=locale)
  return json.loads(json.dumps(fake.format(formatter=provider,**parameters), default=str))
$$;

    
--verify English & Chinese fake names
    select fake_py('en_US','name',null)::varchar as FAKE_NAME from table(generator(rowcount => 10));
    select fake_py('zh_CN','name',null)::varchar as FAKE_NAME from table(generator(rowcount => 10));


In [ ]:
--set context
    use role finservam_admin; use warehouse finservam_devops_wh; use schema finservam.public;

--Set Variables
    set limit_trader = 300;        //on xxlarge will create ~3B trades
    set limit_pm = $limit_trader / 10;   //Every Portfolio Manager (PM) will have about 10 traders reporting to her.

    
--Portfolio Manager (PM)
    --unique number generator
    create or replace sequence pm_id;        
    
    create or replace transient table pm
        comment = 'PM is the Portfolio Manager who manages the traders' as
    select
        fake_py('en_UK','name',null)::varchar as PM,
        pm_id.nextval id
    from table(generator(rowcount => $limit_pm));


In [ ]:
df_pm = session.table('pm')
df_pm.show()

In [ ]:
--trader
--we don't need a transaction but we demo it
begin transaction;
    create or replace transient table trader 
        comment = 'Trader with their Portfolio Manager (PM) and trader authorized buying power' as
    with cte as
    (
    select
        fake_py('en_US','name',null)::varchar as trader,
        uniform(1, $limit_pm, random()) PM_id,                //random function to assign a PM to a trader
        uniform(500, 3500, random())::number buying_power    //how much a trader can buy per day
    from table(generator(rowcount => $limit_trader))
    )
    select
        t.trader,
        pm.pm,
        t.buying_power
    from cte t
    inner join pm on t.pm_id = pm.id
    order by 2,1;

    comment on column public.trader.PM is 'Portfolio Manager (PM) manages traders';
    comment on column public.trader.buying_power is 'Trader is authorized this buying power in each transaction';
commit;



In [ ]:
df_trader=session.table('trader')
df_trader.show()

In [ ]:
--create billions of trades where action = buy
drop table if exists trade;

--size up to save time since we are generating billions of trades
alter warehouse finservam_devops_wh set warehouse_size = 'xxxlarge' wait_for_completion = TRUE   enable_query_acceleration = true;

--create billions of trades where action = buy
create or replace transient table trade
    comment = 'trades made and cash used; unique_key: symbol, exchange, date'
as
 select
      c.*,
      round(buying_power/close,0) num_shares, 
      close * round(buying_power/close,0) * -1 cash,
      t.trader, t.PM
 from
 (
    select
        date, h.symbol, h.exchange, 'buy'::varchar(25) action, close
    from stock_history h
    where year(date) < 2021
 ) c
 full outer join public.trader t
 order by 8,2,1;--Trader, symbol, date

--create billions of trades where action = hold
insert into trade
--hold action
 select
      c.*,
      0 num_shares, 
      0 cash,
      t.trader, t.PM
 from
 (
    select
        date, h.symbol, h.exchange, 'hold'::varchar(25) action, close
    from stock_history h
    where year(date) >= 2021
 ) c
 full outer join public.trader t
order by 8,2,1;--Trader, symbol, date

--size down immediately save credits
alter warehouse finservam_devops_wh suspend;
alter warehouse finservam_devops_wh set warehouse_size = 'xsmall';


In [ ]:
df_trades = session.table('trade')
df_trades.show()

In [ ]:
--create clustered key based on what we sorted
    alter table trade cluster by (trader, symbol, date);

--cluster_by column
    show tables like 'trade';

In [ ]:
//notice Common-Table Expressions (CTEs) and window functions for real-time number of stock shares, cash used, and Profit and Loss
create or replace view public.position
(
  symbol, exchange, date, trader, pm, num_shares_cumulative, cash_cumulative, close, market_value,
  PnL comment 'Profit and Loss: Demonstrate comment on view column'
)
comment = 'what assets owned; demo Window Function running sum'
as
with cte as
(
  select 
      t.symbol, t.exchange, t.date, trader, pm,
      Sum(num_shares) OVER(partition BY t.symbol, t.exchange, trader ORDER BY t.date rows UNBOUNDED PRECEDING ) num_shares_cumulative,
      Sum(cash) OVER(partition BY t.symbol, t.exchange, trader ORDER BY t.date rows UNBOUNDED PRECEDING ) cash_cumulative,
      s.close
  from public.trade t
  inner join public.stock_history s on t.symbol = s.symbol and s.date = t.date
)
select 
*,
num_shares_cumulative * close as market_value, 
(num_shares_cumulative * close) + cash_cumulative as PnL
from cte;


--business_date function to get current date used by the business 
CREATE or replace FUNCTION business_date()
  RETURNS date
  MEMOIZABLE
  AS
  $$
    select max(date) dt from public.trade
  $$;


In [ ]:
df_position=session.table('position')
df_position.show()

In [ ]:
--context
use role finservam_admin; use warehouse finservam_devops_wh; use schema finservam.public;
alter warehouse finservam_devops_wh set warehouse_size = 'xsmall';

--ANSI-Compliant, ie information_schema for data dictionary
select table_type object_type, table_name object_name, comment /* JSON */
from information_schema.tables
where table_schema = 'PUBLIC' and comment is not null
    union all
select 'COLUMN' object_type, table_name || '.' || column_name object_type, comment
from information_schema.columns
where table_schema = 'PUBLIC' and comment is not null
order by 1,2;

In [ ]:
--instant rowcount and metadata
show tables like 'trade';

In [ ]:
--let's format that rowcount
set q = last_query_id();
select "rows", TO_VARCHAR("rows", '999,999,999,999') "pretty_rowcount" from table(result_scan($q));

In [ ]:
--test against a random trader
set trader = (select top 1 trader from trader sample(1) where trader is not null);

//trade - date and quantity of buy, sell, or hold action on assets
select * 
from trade 
where trader = $trader and symbol = 'CMG'
order by date;  

//Notice the Solid State Drive (SSD cache) by running this query and - before the Virtual Warehouse suspends - rerun the query but change the date to something after your original date filter.  [Because that data is already in the Compute layer's memory, it doesnt need to pull it from the micro-partitions again].

In [ ]:
select * 
from trade 
where trader = $trader and symbol = 'SNOW' and action = 'buy' and date >= '2020-10-01'
order by date;  

In [ ]:
//Window functions so we only pay storage for trade table while getting real-time calculations
select get_ddl('view','position'); 

//time-series: what are my real-time trades, cash, and PnL?
--notice query results cache on 2nd execution
select symbol, date, trader, cash_cumulative, num_shares_cumulative, close, market_value, PnL
from position where trader = $trader and date = business_date()
order by pnl desc;


In [ ]:
select symbol, date, trader, cash_cumulative, num_shares_cumulative, close, market_value, PnL
from position where symbol = 'COST' and trader = $trader
order by date;


In [ ]:
--metadata cache for count, min, max, etc
select count(*) from trade;

//Python Faker: ie verify English & Chinese Synthetic names
select fake_py('en_US','name',null)::varchar as FAKE_NAME from table(generator(rowcount => 10));
select fake_py('zh_CN','name',null)::varchar as FAKE_NAME from table(generator(rowcount => 10));

//Cross-Database Joins 
select *
from Financial__Economic_Essentials.cybersyn.stock_price_timeseries s
inner join finservam.public.stock_history h on s.ticker = h.symbol and h.date = s.date
where s.ticker = 'SNOW' and s.variable = 'post-market_close' and s.date = business_date();
